In [3]:
import numpy as np
from collections import defaultdict

"""
Лабораторная работа 2:
1. Посмотроить алгоритм произведения матриц через MapReduce
2. Построить алгоритм линейной регрессии через MapReduce
"""


# Mapper для матрицы A
def mapper_A(matrix_A, p):
    m, n = matrix_A.shape  # C[i,k] = ∑ A[i,j] * B[j,k]
    for i in range(m):
        for j in range(n):
            for k in range(p):
                yield (i, k), ('A', j, matrix_A[i, j])


# Mapper для матрицы B
def mapper_B(matrix_B, m):
    n, p = matrix_B.shape
    for j in range(n):
        for k in range(p):
            for i in range(m):
                yield (i, k), ('B', j, matrix_B[j, k])


# Shuffle — группируем по ключу
def shuffler(mapper_output, intermediate):
    for key, value in mapper_output:
        intermediate[key].append(value)


# Reducer — вычисление одного C[i,k]
def reducer(key, intermediate):
    values = intermediate[key]
    A_vals = {}
    B_vals = {}

    for matrix_type, j, val in values:
        if matrix_type == 'A':
            A_vals[j] = val
        else:
            B_vals[j] = val

    result = 0
    for j in A_vals:
        if j in B_vals:
            result += A_vals[j] * B_vals[j]

    return key, result


# Основная функция MapReduce умножения матриц
def multiply_mapreduce(matrix_A, matrix_B):
    m, n1 = matrix_A.shape
    n2, p = matrix_B.shape

    if n1 != n2:
        raise ValueError("Несовместимые размеры матриц")

    intermediate = defaultdict(list)

    # MAP A
    mapper_a_output = mapper_A(matrix_A, p)
    for item in mapper_a_output:
        shuffler([item], intermediate)

    # MAP B
    mapper_b_output = mapper_B(matrix_B, m)
    for item in mapper_b_output:
        shuffler([item], intermediate)

    # REDUCE
    result_matrix = np.zeros((m, p))
    for key in intermediate:
        i, k = key
        _, value = reducer(key, intermediate)
        result_matrix[i, k] = value

    return result_matrix


def test_matrix_multiplication():
    print("=== Тестирование произведения матриц ===")

    # A = np.array([[1, 2], [3, 4], [5, 6]])
    # B = np.array([[7, 8], [9, 10]])
    A = np.array([
        [123, 845, 392, 711, 456, 980, 234, 567, 890, 321, 654, 777],
        [901, 234, 678, 145, 890, 321, 456, 789, 210, 654, 987, 111],
        [345, 678, 912, 404, 567, 890, 111, 222, 333, 444, 555, 666],
        [789, 245, 678, 901, 432, 156, 777, 888, 999, 101, 202, 303],
        [654, 321, 987, 543, 210, 876, 404, 505, 606, 707, 808, 909],
        [432, 765, 198, 654, 987, 321, 900, 800, 700, 600, 500, 400],
        [111, 222, 333, 444, 555, 666, 123, 234, 345, 456, 567, 678],
        [777, 888, 999, 101, 202, 303, 789, 890, 901, 123, 234, 345],
        [404, 505, 606, 707, 808, 909, 456, 567, 678, 789, 890, 901],

        [912, 134, 256, 378, 490, 512, 634, 756, 878, 990, 111, 222],
        [333, 444, 555, 666, 777, 888, 999, 101, 202, 303, 404, 505],
        [606, 707, 808, 909, 123, 234, 345, 456, 567, 678, 789, 890],
        [321, 654, 987, 210, 543, 876, 135, 246, 357, 468, 579, 680],
        [890, 678, 456, 234, 123, 345, 567, 789, 901, 112, 223, 334],
        [445, 556, 667, 778, 889, 990, 101, 202, 303, 404, 505, 606],
        [707, 808, 909, 111, 222, 333, 444, 555, 666, 777, 888, 999],
        [135, 246, 357, 468, 579, 680, 791, 802, 913, 124, 235, 346],
        [457, 568, 679, 780, 891, 902, 113, 224, 335, 446, 557, 668],
    ])

    B = np.array([
        [912, 345, 678, 123, 456, 789, 234, 567, 890, 321, 654, 987],
        [234, 567, 890, 321, 654, 987, 111, 222, 333, 444, 555, 666],
        [111, 999, 222, 888, 333, 777, 444, 666, 555, 101, 202, 303],
        [444, 666, 555, 101, 202, 303, 808, 707, 606, 505, 404, 303],
        [808, 707, 606, 505, 404, 303, 900, 800, 700, 600, 500, 400],
        [900, 800, 700, 600, 500, 400, 123, 234, 345, 456, 567, 678],

        [321, 432, 543, 654, 765, 876, 987, 109, 210, 321, 432, 543],
        [654, 543, 432, 321, 210, 109, 987, 876, 765, 654, 543, 432],
        [135, 246, 357, 468, 579, 680, 791, 802, 913, 124, 235, 346],
        [457, 568, 679, 780, 891, 902, 113, 224, 335, 446, 557, 668],
        [999, 888, 777, 666, 555, 444, 333, 222, 111, 101, 202, 303],
        [404, 505, 606, 707, 808, 909, 123, 234, 345, 456, 567, 678],
    ])

    res_mr = multiply_mapreduce(A, B)
    res_np = A @ B

    print("MapReduce:\n", res_mr)
    print("NumPy:\n", res_np)
    print("Совпадает:", np.allclose(res_mr, res_np))


test_matrix_multiplication()


=== Тестирование произведения матриц ===
MapReduce:
 [[3599583. 4276450. 4166124. 3511407. 3647287. 4139445. 3246256. 3250088.
  3440240. 2653614. 3069088. 3484562.]
 [4044593. 4084507. 3755826. 3340408. 3195323. 3667950. 3306634. 3204864.
  3335734. 2345605. 2843025. 3340445.]
 [3265227. 4128250. 3641981. 3417580. 3226203. 3900404. 2416097. 2753158.
  2925042. 2233835. 2721283. 3208731.]
 [3077057. 3572093. 3345876. 2868788. 2986549. 3770422. 4198361. 3774561.
  3977680. 2373529. 2778868. 3184207.]
 [4019582. 4780502. 4302638. 4188612. 4048188. 4822067. 3199678. 3379185.
  3679597. 2598348. 3289880. 3981412.]
 [3813644. 4152970. 4325994. 3488414. 3851692. 4353772. 4245230. 3573034.
  3781684. 2966576. 3336522. 3706468.]
 [2722950. 3087687. 2847807. 2635002. 2565396. 2781747. 2087625. 2129502.
  2214297. 1833858. 2111793. 2389728.]
 [2894386. 3719687. 3538440. 3280805. 3351628. 4408042. 3528739. 3355411.
  3704177. 2237826. 2863648. 3489470.]
 [4561167. 5165565. 4884092. 4417474. 44254

In [2]:
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LinearRegression


def mapper_xtx_xty(X, y):
    Xb = np.column_stack([np.ones(X.shape[0]), X])
    XTX_local = np.dot(Xb.T, Xb)
    XTy_local = np.dot(Xb.T, y)
    return [('summary', (XTX_local, XTy_local))]


def shuffle(mapped_data):
    grouped = defaultdict(list)
    for key, value in mapped_data:
        grouped[key].append(value)
    return grouped


def reducer(key, values):
    XTX_total = None
    XTy_total = None
    for XTX, XTy in values:
        if XTX_total is None:
            XTX_total = XTX
            XTy_total = XTy
        else:
            XTX_total += XTX
            XTy_total += XTy
    return XTX_total, XTy_total


def fit_normal_equation(X, y):
    # отправили данные в mapper
    mapped = mapper_xtx_xty(X, y)

    # сгруппировали
    grouped = shuffle(mapped)

    # reducer получил итоговые XᵀX и Xᵀy
    XTX, XTy = reducer('summary', grouped['summary'])

    # решение системы XᵀX * θ = Xᵀy
    try:
        theta = np.linalg.inv(XTX).dot(XTy)
    except:
        # если матрица вырожденная — берём псевдообратную
        theta = np.linalg.pinv(XTX).dot(XTy)

    return theta


def predict(X, theta):
    Xb = np.column_stack([np.ones(X.shape[0]), X])
    return np.dot(Xb, theta)


def test_linear_regression():
    np.random.seed(42)
    X = np.random.randn(1000, 2)
    true_theta = np.array([2, 3, -1])
    y = true_theta[0] + np.dot(X, true_theta[1:]) + 0.1 * np.random.randn(1000)

    print("Истинные коэффициенты:", true_theta)

    # наше решение
    theta = fit_normal_equation(X, y)
    print("\n1. Нормальное уравнение:")
    print("Найденные коэффициенты:", theta)
    print("Средняя ошибка:",
          np.mean(np.abs(theta - true_theta)))

    # sklearn
    print("\n3. Сравнение с sklearn:")
    lr = LinearRegression()
    lr.fit(X, y)
    theta_sklearn = np.concatenate([[lr.intercept_], lr.coef_])
    print("Sklearn коэффициенты:", theta_sklearn)
    print("Средняя ошибка:",
          np.mean(np.abs(theta_sklearn - true_theta)))


test_linear_regression()


Истинные коэффициенты: [ 2  3 -1]

1. Нормальное уравнение:
Найденные коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.0016284438112461608

3. Сравнение с sklearn:
Sklearn коэффициенты: [ 2.00039354  3.0027767  -0.99828491]
Средняя ошибка: 0.0016284438112466788
